# Debug Drill: The Accuracy Trap

**Scenario:**
A colleague built a fraud detection model. They're celebrating.

"99.5% accuracy!" they announce. "Best model I've ever built!"

But the fraud team is angry: "We're catching fewer fraudsters than before!"

**Your Task:**
1. Run the model and verify the high accuracy
2. Investigate why the fraud team is upset
3. Find the right metric and fix the threshold
4. Write a 3-bullet postmortem

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)

np.random.seed(42)

In [ ]:
# Generate imbalanced fraud data (0.5% fraud rate)
n_samples = 10000
fraud_rate = 0.005  # Very rare!

# Features
amount = np.random.exponential(100, n_samples)
hour = np.random.randint(0, 24, n_samples)
is_foreign = np.random.binomial(1, 0.1, n_samples)
customer_age_days = np.random.uniform(1, 1000, n_samples)

# Generate fraud labels (fraudsters tend to: high amount, foreign, night time, new customers)
fraud_prob = 1 / (1 + np.exp(-(
    -7 +
    0.01 * amount +
    2 * is_foreign +
    0.1 * ((hour < 6) | (hour > 22)).astype(int) -
    0.003 * customer_age_days
)))
fraud = (np.random.random(n_samples) < fraud_prob).astype(int)

df = pd.DataFrame({
    'amount': amount,
    'hour': hour,
    'is_foreign': is_foreign,
    'customer_age_days': customer_age_days,
    'fraud': fraud
})

print(f"Dataset: {len(df)} transactions")
print(f"Fraud rate: {df['fraud'].mean():.2%} ({df['fraud'].sum()} fraudulent)")

In [ ]:
# ===== COLLEAGUE'S CODE =====

X = df[['amount', 'hour', 'is_foreign', 'customer_age_days']]
y = df['fraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Using default threshold of 0.5
y_pred = model.predict(X_test)  # <-- BUG: Default threshold for imbalanced data!

accuracy = accuracy_score(y_test, y_pred)

print("=== Colleague's Report ===")
print(f"\n🎉 Accuracy: {accuracy:.1%}")
print("\n✅ Deploying to production...")

---

## Your Investigation

The fraud team says the model is useless. Let's find out why.

### Step 1: The naive baseline

In [ ]:
# What if we predicted NO FRAUD for everyone?
naive_pred = np.zeros_like(y_test)
naive_accuracy = accuracy_score(y_test, naive_pred)

print("=== The Accuracy Trap ===")
print(f"\nColleague's model accuracy: {accuracy:.1%}")
print(f"Naive 'no fraud' accuracy:  {naive_accuracy:.1%}")
print(f"\n❌ Predicting 'no fraud' for EVERYONE gets {naive_accuracy:.1%} accuracy!")
print("   Accuracy is USELESS when fraud is rare.")

In [ ]:
# Check the confusion matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print("=== Confusion Matrix ===")
print(f"\n              Predicted")
print(f"              No Fraud    Fraud")
print(f"Actual No    TN={tn:<6}  FP={fp}")
print(f"Actual Yes   FN={fn:<6}  TP={tp}")

print(f"\n🚨 PROBLEM: We're catching only {tp} out of {tp + fn} fraudsters!")
print(f"   Recall = {tp}/{tp + fn} = {recall_score(y_test, y_pred):.1%}")

### Step 2: Look at the right metrics

In [ ]:
# Get probabilities instead of predictions
y_prob = model.predict_proba(X_test)[:, 1]

print("=== Metrics That Actually Matter ===")
print(f"\nPrecision: {precision_score(y_test, y_pred):.1%}")
print(f"Recall:    {recall_score(y_test, y_pred):.1%} ← This is the problem!")
print(f"F1 Score:  {f1_score(y_test, y_pred):.3f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, y_prob):.3f}")

print(f"\n💡 The model has learned something (AUC > 0.5), but the threshold is wrong!")

### Step 3: TODO - Find the right threshold

In [ ]:
# Business context:
# - Missing fraud (FN) costs $1000 (chargebacks + reputation)
# - False alarm (FP) costs $10 (manual review)

COST_FN = 1000
COST_FP = 10

# Theoretical optimal threshold
theoretical_optimal = COST_FP / (COST_FP + COST_FN)
print(f"Theoretical optimal threshold: {theoretical_optimal:.3f}")
print(f"Colleague's threshold: 0.5")
print(f"\n❌ 0.5 is WAY too high when FN costs 100x more than FP!")

In [ ]:
# TODO: Sweep thresholds and find the cost-optimal one

# Uncomment and complete:

# thresholds = np.arange(0.001, 0.5, 0.005)
# results = []
# 
# for thresh in thresholds:
#     preds = (y_prob >= thresh).astype(int)
#     cm_t = confusion_matrix(y_test, preds)
#     tn_t, fp_t, fn_t, tp_t = cm_t.ravel()
#     
#     cost = fn_t * COST_FN + fp_t * COST_FP
#     recall = recall_score(y_test, preds)
#     precision = precision_score(y_test, preds, zero_division=0)
#     
#     results.append({
#         'threshold': thresh,
#         'cost': cost,
#         'recall': recall,
#         'precision': precision,
#         'fn': fn_t,
#         'fp': fp_t
#     })
# 
# results_df = pd.DataFrame(results)
# optimal_idx = results_df['cost'].idxmin()
# optimal_threshold = results_df.loc[optimal_idx, 'threshold']
# 
# print(f"=== Cost-Optimal Threshold: {optimal_threshold:.3f} ===")
# print(f"Recall at optimal: {results_df.loc[optimal_idx, 'recall']:.1%}")
# print(f"Precision at optimal: {results_df.loc[optimal_idx, 'precision']:.1%}")

In [ ]:
# TODO: Compare original vs optimized

# Uncomment:

# y_pred_fixed = (y_prob >= optimal_threshold).astype(int)
# 
# original_cost = fn * COST_FN + fp * COST_FP
# optimal_cost = results_df.loc[optimal_idx, 'cost']
# 
# print("=== Comparison ===")
# print(f"\n                    Original (0.5)    Optimized ({optimal_threshold:.3f})")
# print(f"  Recall:           {recall_score(y_test, y_pred):>10.1%}      {recall_score(y_test, y_pred_fixed):>10.1%}")
# print(f"  Precision:        {precision_score(y_test, y_pred):>10.1%}      {precision_score(y_test, y_pred_fixed):>10.1%}")
# print(f"  Total Cost:       ${original_cost:>10,}      ${optimal_cost:>10,}")
# print(f"\n  💰 Savings: ${original_cost - optimal_cost:,}")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# assert optimal_threshold < 0.5, "Optimal threshold should be below 0.5 for costly FN"
# assert recall_score(y_test, y_pred_fixed) > recall_score(y_test, y_pred), "Should improve recall"
# assert optimal_cost < original_cost, "Should reduce total cost"
# 
# print("✓ Accuracy trap escaped!")
# print(f"✓ Threshold: 0.5 → {optimal_threshold:.3f}")
# print(f"✓ Recall: {recall_score(y_test, y_pred):.1%} → {recall_score(y_test, y_pred_fixed):.1%}")

### Step 4: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Accuracy Trap

### What happened:
- (Your answer: Why was 99.5% accuracy misleading?)

### Root cause:
- (Your answer: What's wrong with using accuracy on imbalanced data?)

### How to prevent:
- (Your answer: What metrics should we use for rare-event classification?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Accuracy is useless for imbalanced data.** Predicting the majority class gets high accuracy.

2. **Use recall when missing positives is costly** (fraud, disease, churn).

3. **Use precision when false alarms are costly** (spam, recommendations).

4. **The default threshold (0.5) is rarely optimal.** Calculate based on business costs.

---

## Metric Selection Guide

| If... | Use... | Why |
|-------|--------|-----|
| Classes are balanced | Accuracy, F1 | Simple and informative |
| Missing positives is costly | Recall, PR-AUC | Catch as many as possible |
| False alarms are costly | Precision | Be confident when you flag |
| Ranking matters | ROC-AUC | Threshold-independent |
| Class is very rare | PR-AUC | ROC-AUC can be misleading |